#**Text Preprocessing (Bonus)**

In [1]:
import pandas as pd
import numpy as np
import nltk
import re

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


##**LOAD DATASET**

In [2]:
from google.colab import drive
drive.mount('/content/drive')


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import pandas as pd

file_path = "/content/drive/MyDrive/Data/IMDB Dataset.csv"
df = pd.read_csv(file_path)

df.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


##**KIỂM TRA & MÔ TẢ DATA**

In [ ]:
df.info()
df['sentiment'].value_counts()
df['review'].str.len().describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


,review
count,50000.000000
mean,1309.431020
std,989.728014
min,32.000000
25%,699.000000
50%,970.000000
75%,1590.250000
max,13704.000000


##**VÍ DỤ TEXT ĐỂ XỬ LÝ XUYÊN SUỐT**

In [ ]:
sample_text = df['review'].iloc[0]
sample_text


"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

##**LÀM SẠCH CƠ BẢN (HTML, chữ hoa, ký tự thừa)**

In [ ]:
import re

clean_text = sample_text.lower()                 # chuyển về chữ thường
clean_text = re.sub(r'<br\s*/?>', ' ', clean_text)  # xóa thẻ <br />
clean_text = re.sub(r'[^a-z\s]', ' ', clean_text)   # xóa ký tự đặc biệt
clean_text = re.sub(r'\s+', ' ', clean_text).strip()

clean_text[:500]


'one of the other reviewers has mentioned that after watching just oz episode you ll be hooked they are right as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the word it is called oz as that is the nickname given to t'

##**TOKENIZATION (CHIA TỪ)**

In [ ]:
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
tokens = word_tokenize(clean_text)
tokens[:30]


['one',
 'of',
 'the',
 'other',
 'reviewers',
 'has',
 'mentioned',
 'that',
 'after',
 'watching',
 'just',
 'oz',
 'episode',
 'you',
 'll',
 'be',
 'hooked',
 'they',
 'are',
 'right',
 'as',
 'this',
 'is',
 'exactly',
 'what',
 'happened',
 'with',
 'me',
 'the',
 'first']

##**LOẠI STOPWORDS**

Sau khi tokenization, văn bản vẫn còn nhiều từ phổ biến như
"the", "of", "is", "that", ...
Những từ này xuất hiện thường xuyên nhưng không mang nhiều ý nghĩa ngữ nghĩa,
do đó cần được loại bỏ.

In [ ]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

filtered_tokens = [
    word for word in tokens
    if word not in stop_words
]

print("Số token ban đầu:", len(tokens))
print("Số token sau khi loại stopwords:", len(filtered_tokens))

filtered_tokens[:30]


Số token ban đầu: 313
Số token sau khi loại stopwords: 162


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['one',
 'reviewers',
 'mentioned',
 'watching',
 'oz',
 'episode',
 'hooked',
 'right',
 'exactly',
 'happened',
 'first',
 'thing',
 'struck',
 'oz',
 'brutality',
 'unflinching',
 'scenes',
 'violence',
 'set',
 'right',
 'word',
 'go',
 'trust',
 'show',
 'faint',
 'hearted',
 'timid',
 'show',
 'pulls',
 'punches']

Sau khi loại bỏ stop words, số lượng token giảm từ 313 xuống còn 162,
tương đương giảm khoảng 48%.

Các token còn lại chủ yếu là các từ mang nhiều thông tin nội dung
và cảm xúc như *brutality*, *violence*, *hooked*,
giúp văn bản phù hợp hơn cho bài toán phân tích cảm xúc.

##**STEMMING & LEMMATIZATION**

Stemming và Lemmatization là hai kỹ thuật đưa từ về dạng gốc
nhằm giảm độ phức tạp của dữ liệu văn bản.

Hai phương pháp này có cách tiếp cận và mức độ chính xác khác nhau

In [ ]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk

nltk.download('wordnet')

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

stemmed_words = [stemmer.stem(word) for word in filtered_tokens]
lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_tokens]

# So sánh 15 từ đầu tiên
list(zip(filtered_tokens[:15], stemmed_words[:15], lemmatized_words[:15]))


[nltk_data] Downloading package wordnet to /root/nltk_data...


[('one', 'one', 'one'),
 ('reviewers', 'review', 'reviewer'),
 ('mentioned', 'mention', 'mentioned'),
 ('watching', 'watch', 'watching'),
 ('oz', 'oz', 'oz'),
 ('episode', 'episod', 'episode'),
 ('hooked', 'hook', 'hooked'),
 ('right', 'right', 'right'),
 ('exactly', 'exactli', 'exactly'),
 ('happened', 'happen', 'happened'),
 ('first', 'first', 'first'),
 ('thing', 'thing', 'thing'),
 ('struck', 'struck', 'struck'),
 ('oz', 'oz', 'oz'),
 ('brutality', 'brutal', 'brutality')]

- **Stemming**:
  - Cắt từ dựa trên các quy tắc đơn giản
  - Thực hiện nhanh
  - Có thể tạo ra từ không đúng ngữ pháp

- **Lemmatization**:
  - Đưa từ về dạng từ điển
  - Giữ được ý nghĩa ngữ cảnh tốt hơn
  - Tốn nhiều tài nguyên tính toán hơn

Trong bài toán phân tích cảm xúc (sentiment analysis),
lemmatization thường được ưu tiên vì giúp bảo toàn ý nghĩa của từ.

##**VECTOR HÓA TOÀN BỘ DATASET**

**Bag of Words (BoW)**

Bag of Words biểu diễn văn bản dựa trên tần suất xuất hiện của từ
mà không xét đến ngữ cảnh.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer(max_features=5000)
X_bow = bow_vectorizer.fit_transform(df['review'])

X_bow.shape


(50000, 5000)

**TF-IDF**

TF-IDF (Term Frequency - Inverse Document Frequency)
giảm ảnh hưởng của các từ xuất hiện quá thường xuyên
và làm nổi bật các từ mang nhiều thông tin hơn.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(df['review'])

X_tfidf.shape


(50000, 5000)

- **Bag of Words**:
  - Dễ hiểu và dễ triển khai
  - Các từ phổ biến có trọng số lớn
  - Không phản ánh mức độ quan trọng của từ

- **TF-IDF**:
  - Giảm ảnh hưởng của các từ phổ biến
  - Làm nổi bật các từ mang nhiều thông tin
  - Phù hợp hơn cho bài toán phân tích cảm xúc

Do đó, TF-IDF thường được ưu tiên sử dụng trong các mô hình sentiment analysis.

##**TỔNG KẾT**

Trong notebook này, các bước tiền xử lý dữ liệu văn bản đã được thực hiện gồm:
- Làm sạch văn bản
- Tokenization
- Loại bỏ stop words
- Stemming và Lemmatization
- Vector hóa văn bản bằng Bag of Words và TF-IDF

Sau khi tiền xử lý, dữ liệu đã sẵn sàng để sử dụng cho các mô hình học máy.